In [36]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision.transforms.functional as TF
import torch.utils.data as data
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from torchvision import transforms
from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine

In [2]:
class MRDataset(data.Dataset):
    def __init__(self, root_dir, task, plane, train=True, transform=None, weights=None):
        super().__init__()
        self.task = task
        self.plane = plane
        self.root_dir = root_dir
        self.train = train
        if self.train:
            self.fold_path = self.root_dir + 'train/{}/'.format(plane)
            self.records = pd.read_csv(self.root_dir + 'train-{}.csv'.format(task), header=None, names=['id', 'label'])
        else:
            transform = None
            self.fold_path = self.root_dir + 'valid/{}/'.format(plane)
            self.records = pd.read_csv(self.root_dir + 'valid-{}.csv'.format(task), header=None, names=['id', 'label'])
            
        self.records['id'] = self.records['id'].map(lambda i: '0'*(4 - len(str(i))) + str(i))
        self.paths = [self.fold_path + filename + '.npy' for filename in self.records['id'].tolist()]
        self.labels = self.records['label'].tolist()
        
        self.transform = transform
        
        if weights is None:
            pos = np.sum(self.labels)
            neg = len(self.labels) - pos
            self.weights = torch.FloatTensor([1, neg/pos])
        else:
            self.weights = torch.FloatTensor(weights)
            
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        array = np.load(self.paths[index])
        label = self.labels[index]
        
        if label == 1:
            label = torch.FloatTensor([0, 1])
        elif label == 0:
            label = torch.FloatTensor([1, 0])
            
        if self.transform:
            array = self.transform(array)
        else:
            array = np.stack((array, )*3, axis=1)
            array = torch.FloatTensor(array)
            
#         if label.item() == 1:
#             weight = np.array([self.weights[1]])
#             weight = torch.FloatTensor(weight)
#         else:
#             weight = np.array([self.weights[0]])
#             weight = torch.FloatTensor(weight)
            
        return array, label, self.weights

In [3]:
class MRNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model = models.alexnet(pretrained=True)
        self.pooling_layer = nn.AdaptiveAvgPool2d(1)
        self.classifer = nn.Linear(256, 2)
        
    def forward(self, x):
        x = torch.squeeze(x, dim=0)
        features = self.pretrained_model.features(x)
        pooled_features = self.pooling_layer(features)
        pooled_features = pooled_features.view(pooled_features.size(0), -1)
        flattened_features = torch.max(pooled_features, 0, keepdim=True)[0]
        output = self.classifer(flattened_features)
        return output

In [5]:
coronal_mrnet = torch.load('coronal_best.pth')
axial_mrnet = torch.load('axial_best.pth')
sagittal_mrnet = torch.load('sagittal_best.pth')

In [22]:
def extract_predictions(model, task, plane, train=True):
    
    train_dataset = MRDataset(root_dir='', task=task, plane=plane, transform=None, train=train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)
    
    predictions = []
    labels = []
    
    if torch.cuda.is_available():
        model.cuda()
    model.eval()
    with torch.no_grad():
        for image, label, _ in tqdm(train_loader):
            logit = model(image.cuda())
            prediction = torch.sigmoid(logit)
            predictions.append(prediction[0].cpu().numpy())
            labels.append(label[0].numpy())
    return predictions, labels

In [23]:
acl_models = {'coronal': coronal_mrnet, 'axial': axial_mrnet, 'sagittal': sagittal_mrnet}

In [24]:
task = 'acl'
results = {}

for plane in acl_models.keys():
    predictions, labels = extract_predictions(model=acl_models[plane], task=task, plane=plane)
    results['labels'] = labels
    results[plane] = np.array(predictions)
    
# x = np.zeros((len(predictions), 3))
# x[:, 0] = results['coronal']
# x[:, 1] = results['axial']
# x[:, 2] = results['sagittal']




 25%|██▌       | 288/1130 [00:27<01:11, 11.75it/s]


 52%|█████▏    | 591/1130 [00:52<00:52, 10.20it/s]


 79%|███████▉  | 893/1130 [01:18<00:20, 11.29it/s]


100%|██████████| 1130/1130 [01:38<00:00, 11.50it/s]

  5%|▍         | 54/1130 [00:05<01:41, 10.62it/s]


 31%|███       | 352/1130 [00:34<01:24,  9.24it/s]


...

100%|██████████| 1130/1130 [01:46<00:00, 10.57it/s]

  9%|▉         | 102/1130 [00:08<01:24, 12.18it/s]


 35%|███▌      | 401/1130 [00:34<00:57, 12.74it/s]


 62%|██████▏   | 703/1130 [00:59<00:36, 11.66it/s]


 89%|████████▉ | 1004/1130 [01:25<00:11, 10.69it/s]


100%|██████████| 1130/1130 [01:36<00:00, 11.76it/s]


In [35]:
x = np.concatenate((results['coronal'], results['axial'], results['sagittal']), axis=1)
print(x.shape) # (1130, 6)
# y = np.array([np.argmax(label) for labe in results['labels']])
y = np.argmax(np.array(results['labels']), axis=1)
print(y.shape)

clf = LogisticRegression()
clf.fit(x, y)
clf.score(x, y)

(1130, 6)
(1130,)


0.8893805309734514

In [40]:
y_pred = clf.predict_proba(x)[:, 1]
roc_auc_score(y, y_pred), clf.score(x, y)

(0.9090762139162355, 0.8893805309734514)

In [42]:
task = 'acl'
results_val = {}

for plane in acl_models.keys():
    predictions, labels = extract_predictions(model=acl_models[plane], task=task, plane=plane, train=False)
    results_val['labels'] = labels
    results_val[plane] = np.array(predictions)
    
x_val = np.concatenate((results_val['coronal'], results_val['axial'], results_val['sagittal']), axis=1)
print(x_val.shape) # (1130, 6)
# y = np.array([np.argmax(label) for labe in results['labels']])
y_val = np.argmax(np.array(results_val['labels']), axis=1)
print(y_val.shape)
    
    
y_pred = clf.predict_proba(x_val)[:, 1]
roc_auc_score(y_val, y_pred), clf.score(x_val, y_val)


100%|██████████| 120/120 [00:07<00:00, 16.95it/s]

100%|██████████| 120/120 [00:08<00:00, 14.80it/s]

 78%|███████▊  | 94/120 [00:05<00:01, 15.74it/s]


100%|██████████| 120/120 [00:07<00:00, 16.78it/s]

(120, 6)
(120,)


(0.878226711560045, 0.75)

In [86]:
clf.predict(x_val)

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0], dtype=int64)

In [104]:
case = '1247'
acl_models = {'coronal': coronal_mrnet, 'axial': axial_mrnet, 'sagittal': sagittal_mrnet}
results_pre = {}

for plane in ['coronal', 'axial', 'sagittal']:
    filename = 'valid/'+plane+'/'+case+'.npy'
    img = np.load(filename)
    img = np.stack((img, )*3, axis=1)
    img = torch.FloatTensor(img)
    if torch.cuda.is_available():
        acl_models[plane].cuda()
    acl_models[plane].eval()
    with torch.no_grad():
        pre = acl_models[plane](img.cuda())
        pre = torch.sigmoid(pre)
        results_pre[plane] = np.array(pre.cpu().numpy())

In [105]:
x_pre = np.concatenate((results_pre['coronal'], results_pre['axial'], results_pre['sagittal']), axis=1)
print(x_pre.shape) # (1, 6)


(1, 6)


In [106]:
clf.predict(x_pre), clf.predict_proba(x_pre)

(array([0], dtype=int64), array([[0.93574139, 0.06425861]]))